In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     

import torchvision
import torchvision.transforms as transforms

from models import *


global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
    
    
batch_size = 128
model_name = "VGG16_2bit_4bit"
model = VGG16_quant_2bit_4bit()

print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    #adjust_list = [30, 60, 90, 120]
    #if epoch in adjust_list:
    #    for param_group in optimizer.param_groups:
    #        param_group['lr'] = param_group['lr'] * 0.1        
    if epoch % 20 == 0:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.3162   

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
VGG_quant(
  (features): Sequential(
    (0): QuantConv2d(
      3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): QuantConv2d(
      64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): QuantConv2d(
      64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): QuantConv2d(
      128, 128, kernel_size=(3, 3), stride

In [ ]:
# This cell won't be given, but students will complete the training

lr = 5e-3
weight_decay = 5e-4
epochs = 200
best_prec = 90.32

#model = nn.DataParallel(model).cuda()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
#cudnn.benchmark = True

if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name)
if not os.path.exists(fdir):
    os.makedirs(fdir)
        
checkpoint = torch.load(fdir + '/model_best.pth.tar')
model.load_state_dict(checkpoint['state_dict'])

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

In [2]:
PATH = "result/VGG16_2bit_4bit/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 9023/10000 (90%)



In [3]:
#send an input and grap the value by using prehook like HW3
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
save_output = SaveOutput()
     
model.features[27].register_forward_pre_hook(save_output) #input to (16, 16) conv
model.features[29].register_forward_pre_hook(save_output) #output to (16,16) conv + ReLU

dataiter = iter(testloader)
images, labels = next(dataiter)
image = images[0].to(device)
out = model(image.reshape(1,3,32,32))

In [4]:
w_bit = model.features[27].bit
weight_q = model.features[27].weight_q # quantized value is stored during the training
w_alpha = model.features[27].weight_quant.wgt_alpha   # alpha is defined in your model already. bring it out here
w_delta = w_alpha/(2**(w_bit-1)-1)    # delta can be calculated by using alpha and w_bit
weight_int = weight_q/w_delta # w_int can be calculated by weight_q and w_delta
print(weight_q) # you should see clean integer numbers

Parameter containing:
tensor([[[[ 0.1921,  0.1921,  0.1921],
          [ 0.1921,  0.1921,  0.1921],
          [-0.1921,  0.1921,  0.1921]],

         [[ 0.1921,  0.1921, -0.1921],
          [-0.1921,  0.1921,  0.1921],
          [ 0.1921,  0.1921, -0.1921]],

         [[-0.1921, -0.1921,  0.1921],
          [-0.1921, -0.1921, -0.1921],
          [-0.1921, -0.1921,  0.1921]],

         ...,

         [[-0.1921, -0.1921, -0.1921],
          [-0.1921,  0.1921,  0.1921],
          [-0.1921, -0.1921,  0.1921]],

         [[-0.0274, -0.0274, -0.0274],
          [-0.0000, -0.0000, -0.0000],
          [-0.0000, -0.0000, -0.0000]],

         [[ 0.1921,  0.1921,  0.1921],
          [ 0.1921,  0.1921, -0.1921],
          [-0.1921,  0.1921, -0.1921]]],


        [[[ 0.1921,  0.1921, -0.1921],
          [ 0.1921, -0.1921,  0.1921],
          [ 0.1921,  0.1921, -0.1921]],

         [[-0.1921,  0.1921,  0.1921],
          [ 0.1921, -0.1921,  0.1921],
          [ 0.1921,  0.1921,  0.1921]],

         

In [5]:
x_bit = model.features[27].x_bit    
x = save_output.outputs[0][0]  # input of conv layer
x_alpha  = model.features[27].act_alpha
x_delta = x_alpha/(2**x_bit - 1)

act_quant_fn = act_quantization(x_bit) # define the quantization function
x_q = act_quant_fn(x, x_alpha)         # create the quantized value for x

x_int = x_q / x_delta
print(x_int) # you should see clean integer numbers 

tensor([[[[3.0000, 3.0000, 2.0000, 1.0000],
          [3.0000, 3.0000, 3.0000, 3.0000],
          [2.0000, 2.0000, 1.0000, 0.0000],
          [0.0000, 1.0000, 0.0000, 1.0000]],

         [[2.0000, 3.0000, 3.0000, 2.0000],
          [3.0000, 3.0000, 2.0000, 1.0000],
          [1.0000, 1.0000, 0.0000, 0.0000],
          [0.0000, 1.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000, 1.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [3.0000, 3.0000, 3.0000, 2.0000],
          [2.0000, 2.0000, 2.0000, 1.0000]],

         [[2.0000, 2.0000, 2.0000, 0.0000],
          [2.0000, 3.0000, 2.0000, 0.0000],
          [3.0000, 2.0000, 3.0000, 1.0000],
          [2.0000, 2.0000, 1.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0

In [6]:
conv_int = torch.nn.Conv2d(in_channels = 16, out_channels=16, kernel_size = 3, padding=1, bias = False)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)

psum_int = conv_int(x_int)    # output_int can be calculated with conv_int and x_int
output_recovered = psum_int*x_delta*w_delta  # recover with x_delta and w_delta
relu = nn.ReLU()
output_recovered = relu(output_recovered)
print(output_recovered.shape)

torch.Size([1, 16, 4, 4])


In [7]:
output_ref = save_output.outputs[1][0]
#print(output_ref)
difference = abs( output_ref - output_recovered )
print(difference.sum())  ## It should be small, e.g.,2.3 in my trainned model

tensor(2.6767e-05, device='cuda:0', grad_fn=<SumBackward0>)


In [8]:
a_int = x_int[0,:,:,:]  # pick only one input out of batch

w_int = torch.reshape(weight_int, (weight_int.size(0), weight_int.size(1), -1))  # merge ki, kj index to kij
print(w_int.shape)
                      
padding = 1
stride = 1
array_size = 8 # row and column number

nig = range(a_int.size(1))  ## ni group
njg = range(a_int.size(2))  ## nj group

icg = range(int(w_int.size(1)))  ## input channel 
ocg = range(int(w_int.size(0)))  ## output channel

ic_tileg = range(int(len(icg)/array_size))
oc_tileg = range(int(len(ocg)/array_size))

kijg = range(w_int.size(2))
ki_dim = int(math.sqrt(w_int.size(2)))  ## Kernel's 1 dim size

######## Padding before Convolution #######
a_pad = torch.zeros(len(icg), len(nig)+padding*2, len(nig)+padding*2).cuda()

a_pad[ :, padding:padding+len(nig), padding:padding+len(njg)] = a_int.cuda()
a_pad = torch.reshape(a_pad, (a_pad.size(0), -1))
# a_pad.size() = [64, (32+2pad)*(32+2pad)]


a_tile = torch.zeros(len(ic_tileg), array_size,    a_pad.size(1)).cuda() 
w_tile = torch.zeros(len(oc_tileg)*len(ic_tileg), array_size, array_size, len(kijg)).cuda() 

for ic_tile in ic_tileg:
    a_tile[ic_tile,:,:] = a_pad[ic_tile*array_size:(ic_tile+1)*array_size,:]

for ic_tile in ic_tileg:
    for oc_tile in oc_tileg:
        w_tile[oc_tile*len(oc_tileg) + ic_tile,:,:,:] = w_int[oc_tile*array_size:(oc_tile+1)*array_size, ic_tile*array_size:(ic_tile+1)*array_size, :]



###########################################

p_nijg = range(a_pad.size(1)) ## psum nij group

psum = torch.zeros(len(ic_tileg), len(oc_tileg), array_size, len(p_nijg), len(kijg)).cuda() 

for kij in kijg:
    for ic_tile in ic_tileg:       # Tiling into array_sizeXarray_size array
        for oc_tile in oc_tileg:   # Tiling into array_sizeXarray_size array        
            for nij in p_nijg:       # time domain, sequentially given input
                    m = nn.Linear(array_size, array_size, bias=False)
                    #m.weight = torch.nn.Parameter(w_int[oc_tile*array_size:(oc_tile+1)*array_size, ic_tile*array_size:(ic_tile+1)*array_size, kij])
                    m.weight = torch.nn.Parameter(w_tile[len(oc_tileg)*oc_tile+ic_tile,:,:,kij])
                    psum[ic_tile, oc_tile, :, nij, kij] = m(a_tile[ic_tile,:,nij]).cuda()
 

torch.Size([16, 16, 9])


In [9]:
import math

a_pad_ni_dim = int(math.sqrt(a_pad.size(1))) 

o_ni_dim = int((a_pad_ni_dim - (ki_dim- 1) - 1)/stride + 1)
o_nijg = range(o_ni_dim**2)    
    
out = torch.zeros(len(ocg), len(o_nijg)).cuda()
  
   
### SFP accumulation ###
for o_nij in o_nijg: 
    for kij in kijg:
        for ic_tile in ic_tileg:    
            for oc_tile in oc_tileg:   
                out[oc_tile*array_size:(oc_tile+1)*array_size, o_nij] = out[oc_tile*array_size:(oc_tile+1)*array_size, o_nij] + \
                psum[ic_tile, oc_tile, :, int(o_nij/o_ni_dim)*a_pad_ni_dim + o_nij%o_ni_dim + int(kij/ki_dim)*a_pad_ni_dim + kij%ki_dim, kij]
                ## 4th index = (int(o_nij/30)*32 + o_nij%30) + (int(kij/3)*32 + kij%3)

In [10]:
out_2D = torch.reshape(out, (out.size(0), o_ni_dim, -1))
difference = (out_2D - psum_int[0,:,:,:])
print(difference.sum())

tensor(-0.0001, device='cuda:0', grad_fn=<SumBackward0>)


In [11]:
### show this cell partially. The following cells should be printed by students ###
bit_precision = x_bit
file = open(f"activation.txt", 'w') #write to file
file.write('#time0row15[msb-lsb],time0row14[msb-lst],....,time0row0[msb-lst]#\n')
file.write('#time1row15[msb-lsb],time1row14[msb-lst],....,time1row0[msb-lst]#\n')
file.write('#................#\n')
for i in range(a_tile.size(2)):  # time step
    for ic_tile in ic_tileg:  
        for j in range(a_tile.size(1)): # row #
            X_num = round(a_tile[1 - ic_tile, 7-j,i].item())
            X_bin = '{0:02b}'.format(X_num)
            for k in range(bit_precision):
                file.write(X_bin[k])        
        #file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close() #close file    


In [12]:
for oc_tile in oc_tileg:
    for kij in kijg:
        file = open(f"weight_octile{oc_tile}_kij{kij}.txt", 'w') #write to file
        file.write('#col0row15[msb-lsb],col0row13[msb-lst],....,col0row1[msb-lst]#\n')
        file.write('#col0row14[msb-lsb],col0row12[msb-lst],....,col0row0[msb-lst]#\n')
        file.write('#................#\n')
        bit_precision = 4
        for i in range(w_tile.size(1)):  # col #output channel
            for rows in [[7,5,3,1], [6,4,2,0]]:#do the even rows first
                for ic_tile in ic_tileg:
                    tile_id = len(oc_tileg)*oc_tile+ 1 - ic_tile
                    print(tile_id)
                    for row in rows:
                        w_val = round(w_tile[tile_id, i, row, kij].item())
                        if w_val < 0:
                            w_val = 16 + w_val
                        W_bin = '{0:04b}'.format(w_val)
                        for k in range(bit_precision):
                            file.write(W_bin[k])        
                        #file.write(' ')
                file.write('\n')
        file.close() #close file 


1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2


In [13]:

oc_tile_id = 0 
for oc_tile in oc_tileg:
    out_tile = out[oc_tile*array_size:(oc_tile+1)*array_size, :]
    bit_precision = 16
    file = open(f"out_octile{oc_tile}.txt", 'w') #write to file
    file.write('#o_nij0col7[msb-lsb],o_nij0col6[msb-lst],....,o_nij0col0[msb-lst]#\n')
    file.write('#o_nij1col7[msb-lsb],o_nij1col6[msb-lst],....,o_nij1col0[msb-lst]#\n')
    file.write('#................#\n')
    for i in range(out_tile.size(1)):  # time step
        for j in range(out_tile.size(0)): # row #
            psum_val = round(out_tile[7-j, i].item())
            if psum_val < 0:
                psum_val = 2**16 + psum_val
            PSUM_bin = '{0:016b}'.format(psum_val)
            for k in range(bit_precision):
                file.write(PSUM_bin[k])         
            #file.write(' ')  # for visibility with blank between words, you can use
        file.write('\n')
    file.close() #close file 